### 0. 事前準備

以下のデータを取得して、このノートブックと同じディレクトリにある`data`ディレクトリに配置します。

__open-access-met Open Access csv__  
  URL:  
    ・https://github.com/metmuseum/openaccess  
  DATA:  
    ・MetObjects.csv  
  関連情報:  
    ・40万点の作品画像を無料開放。メトロポリタン美術館がAPIを公開  
  　https://bijutsutecho.com/magazine/news/headline/18741

---

ローカル環境でjupyterを起動している場合は、以下のツールを取得して、このノートブックと同じディレクトリにある`lib`ディレクトリに配置します。  

__ChromeDriver__  
  URL:  
    ・https://chromedriver.chromium.org/  
  TOOL:  
    ・chromedriver.exe（Windows用）/ chromedriver（Mac用）/ chromedriver（Linux用）    
  注意:  
    ・OSにChromeまたはChromiumブラウザがインストールされていることが前提となります。  
    ・PCに現在インストールされているChromeと同じバージョンのものを選択してください。

In [ ]:
%matplotlib inline

from bs4 import BeautifulSoup
import glob
from IPython.core.display import HTML
from IPython.display import Image, display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import seaborn as sns
from selenium import webdriver
from shutil import which

ノートブックのレイアウト定義。

In [ ]:
%%html
<style>
  table {margin-left: 0 !important;} # markdownの表を左寄せにする。
</style>

### 1. データの概要を掴む

作品データをロード。

In [ ]:
met_object_df = pd.read_csv('data/MetObjects.csv')

len(met_object_df)

幾つかの列でデータ型が混じってしまっていて正しく読み込めないようです。

どんな列で問題が発生しているのかを確認します。

In [ ]:
met_object_df.iloc[:,[5,7,10,11,12,13,14,34,35,36,37,38,39,40,41,42,43,44,45,46]]

とりあえず今回使いそうな２つの列、`AccessionYear`（取得年）と`Country`（国）だけはデータ型を指定して読み込み直します。

In [ ]:
met_object_df = pd.read_csv('data/MetObjects.csv', dtype={"AccessionYear": "string", "Country": "string"})

len(met_object_df)

変換に失敗しそうですが、`AccessionYear`をTimestamp型に変換してみます。

In [ ]:
try:
    met_object_df['AccessionYear'] = pd.to_datetime(met_object_df['AccessionYear'])
except Exception as e:
    print(f"error occured: '{e}'")

変換に失敗した行を確認。

In [ ]:
met_object_df[met_object_df['AccessionYear'] == '19171917']

データが間違っているようなので修正してしまいます。

In [ ]:
met_object_df.loc[473803,'AccessionYear'] = '1917'

met_object_df.loc[473803,'AccessionYear']

もう一度変換して、そのまま列に格納します。

In [ ]:
met_object_df['AccessionYear'] = pd.to_datetime(met_object_df['AccessionYear'])

エラーが発生しないので問題なさそうですが、エラーのあったデータがどう変換されたか確認します。

In [ ]:
met_object_df.loc[473803,'AccessionYear']

データの中身を確認。

In [ ]:
met_object_df.head()

データに抜けが多いようなので、`info()`で状況を確認。

`AccessionYear`は`datetime64[ns]`型になっています。

In [ ]:
met_object_df.info()

今回は以下の列のデータを利用します。  
`Artist Display Name`と`Country`は抜けが多いので注意が必要。

|#|Column|Non-Null Count|備考|
|:---:|:---|---|:---|
|0|Object Number|477390||
|1|Is Highlight|477390|重要作品であるか？|
|3|Is Public Domain|477390||
|4|Object ID|477390||
|7|AccessionYear|472891|取得年|
|8|Object Name|475700||
|9|Title|448204||
|18|Artist Display Name|275116||
|29|Object Begin Date|477390|製作開始日|      
|30|Object End Date|477390|製作完了日|
|33|Credit Line|476933|提供者| 
|38|Country|75746||
|47|Link Resource|477390|詳細ページURL|

In [ ]:
met_object_df = met_object_df.iloc[:,[0, 1, 3, 4, 7, 8, 9, 18, 29, 30, 33, 38, 47]]

met_object_df.head()

ちなみに、`Object Begin Date`と`Object End Date`が`int`型になっていますが、これは`datetime64`型（`Timestamp`型）の範囲を超えてしまっているので変換はできません。

参考）  
__pandas out of bounds nanosecond timestamp after offset rollforward plus adding a month offset__  
https://stackoverflow.com/questions/32888124/pandas-out-of-bounds-nanosecond-timestamp-after-offset-rollforward-plus-adding-a

In [ ]:
print("最小値:", pd.Timestamp.min)
print("最大値:", pd.Timestamp.max)

これはOK。

In [ ]:
pd.to_datetime('1680-01-01') # or pd.Timestamp('1680-01-01')

これはダメ。

In [ ]:
# 変換できない場合は「NaT」を返す。
pd.to_datetime('1677-01-01', errors="coerce") # or pd.Timestamp('1677-01-01')

国別の作品数を確認。

In [ ]:
met_object_df['Country'].value_counts().head(30)

製作完了日の分布をグラフで確認。

In [ ]:
fig, ax = plt.subplots(figsize=(16,4))

sns.histplot(met_object_df['Object End Date'], kde=False)

グラフがちょっとおかしいです。

とりあえず統計量を確認します。

In [ ]:
met_object_df['Object End Date'].describe()

最大値も少しおかしいですが、最小値がとんでもない値になっています。

どんな作品なのか、実際にサイトにアクセスして確認します。

In [ ]:
oldest_work_links = met_object_df.query('`Object End Date` == -240_000')['Link Resource'].values
for link in oldest_work_links:
    print(link)

石器ですね… 桁外れの外れ値のせいで紀元後の分布が全く把握できません。  
今回は石器を対象外にするとして紀元前2500年〜2022年に限定し、グラフを出力します。

どんなことが読み取れますか？

In [ ]:
fig, ax = plt.subplots(figsize=(16,4))

sns.histplot(met_object_df.query('-2500 <= `Object End Date` <= 2022')['Object End Date'], kde=False)
ax.set_xlim(-2500, 2022)

さらに作品の多い1400年以降に絞って見てみましょう。

どんなことが読み取れますか？

In [ ]:
fig, ax = plt.subplots(figsize=(16,4))

ax = sns.histplot(met_object_df['Object End Date'][met_object_df['Object End Date'] >= 1400], kde=False)
ax.set_xlim(1400, 2022)

### 2. groupbyによる集約

ここでは、`groupby()`を利用してデータを集約してみます。

手始めに`Country`で集約してみましょう。

`DataFrameGroupBy`オブジェクトに対し、様々な集約関数を呼び出します。

In [ ]:
met_objects_groupby_country = met_object_df.groupby('Country')

met_objects_groupby_country

#### 2-1. 簡単な集計関数の利用

まずは`size()`で国別の件数を取得します。

In [ ]:
country_size = met_objects_groupby_country.size()

country_size

似たような関数に`count()`がありますが、これは列ごとの件数を取得します。

また、`size()`は行数を取得する一方で、`count()`はNA、つまり値がない場合は含めません。

In [ ]:
met_objects_groupby_country.count()

国別の`Object End Date`の最大値はどうでしょう？

In [ ]:
met_objects_groupby_country['Object End Date'].max()

国別の`Object End Date`の平均値は？

In [ ]:
met_objects_groupby_country['Object End Date'].mean()

国別の`Object End Date`の中央値は？

面倒になってきました…

In [ ]:
met_objects_groupby_country['Object End Date'].median()

実はここでも`describe()`が使えます。

In [ ]:
met_objects_groupby_country['Object End Date'].describe()

#### 2-2. agg関数の利用

複数の任意の集約関数を利用したい場合、`agg()`関数を使って集約関数をまとめて指定できます。

In [ ]:
met_objects_groupby_country.agg({'AccessionYear': ['min', 'max', 'mean', 'median']})

引数の`dict`のキーとして複数の列を対象にすることも可能です。

In [ ]:
met_objects_groupby_country.agg({'AccessionYear': ['min', 'max'], 'Object Begin Date': ['mean', 'median']})

`agg()`は引数に

`結果列の名前=('対象の列', '集計関数')`

を必要な分だけ指定することもできます。

In [ ]:
met_objects_groupby_country.agg(
    AY_mean=('AccessionYear', 'mean'),
    OBD_min=('Object Begin Date', 'min'),
    OED_max=('Object End Date', 'max'))

#### 2-3. apply関数の利用

任意の関数を集約単位で実行したい場合、`apply()`を利用します。

ここでは作品データを`Object Name`で集約することにします。

まずはどんな値があるか確認。

In [ ]:
met_object_df['Object Name'].value_counts().head(20)

`groupby()`で`DataFrameGroupBy`オブジェクトを生成します。

In [ ]:
met_objects_groupby_object_name = met_object_df.groupby('Object Name')

met_objects_groupby_object_name

ここではジャンルごとの重要度率を取得してみましょう。

まず、データの重要作品率を取得する関数は以下のように定義できるでしょう。

In [ ]:
def get_highlight_rate(object_df):
    """重要作品率を取得。

    Args:
        object_df (pandas.DataFrame): 美術品データ。
    Returns:
        numpy.float64: 重要作品率。
    """

    return round(object_df['Is Highlight'].sum() / len(object_df), 2)

実際にデータ全体に対して実行してみましょう。

In [ ]:
get_highlight_rate(met_object_df)

この関数を`apply()`に引数として渡すことで、集約単位ごとに実行することができます。

実際に`Object Name`ごとの重要作品率を取得し、降順で表示してみましょう。

In [ ]:
met_objects_groupby_object_name.apply(get_highlight_rate).sort_values(ascending=False).head(20)

重要作品率が100%のものばかりになってしまいました。

おそらくはジャンル別の作品点数が少ないものがあるのでしょう、今回は500点以上あるジャンルのみを対象とします。

In [ ]:
popular_objects = met_objects_groupby_object_name.filter(lambda x: len(x) >= 500)

print(f"全作品データ数: {len(met_object_df):,}")
print(f"500点以上あるジャンル限定: {len(popular_objects):,}")

改めて重要作品率を取得します。

どんなことが読み取れますか？

In [ ]:
popular_objects.groupby('Object Name').apply(get_highlight_rate).sort_values(ascending=False).head(20)

### 3. 横持ちへの変換

ここでは、データの各行に１つずつの情報を持っている縦持ちの状態から、クロス集計表のような横持ちの状態への変換をおこないます。

縦軸を作品の提供者、横軸を作品ジャンルとするクロス集計表を作ってみましょう。

提供者は`Credit Line`です。

In [ ]:
print(met_object_df['Credit Line'].head(30))

print(f"データ数: {len(met_object_df['Credit Line']):,}")

提供者の名前と提供年が含まれてしまっていますので、データを分離して２つの列に格納します。

In [ ]:
met_object_df[['credit_line_name', 'credit_line_year']] = \
    met_object_df['Credit Line'].str.extract("(.+), (\d{4})", expand=True)

met_object_df.head()

提供者がどの位いるのか確認。

In [ ]:
met_object_df['credit_line_name'].value_counts()

表を作るには多すぎるので上位10までに絞ります。

In [ ]:
popular_credit_line_name_10 = met_object_df['credit_line_name'].value_counts()[:10].index.values

print("\n".join(popular_credit_line_name_10))

ジャンルも上位10までに絞ります。

In [ ]:
popular_object_name_10 = met_object_df['Object Name'].value_counts()[:10].index.values

print("\n".join(popular_object_name_10))

In [ ]:
met_object_popular_credit_line_df = met_object_df[
    met_object_df['credit_line_name'].isin(popular_credit_line_name_10) &
    met_object_df['Object Name'].isin(popular_object_name_10)]

print(f"全作品数: {len(met_object_df):,}")
print(f"絞った数: {len(met_object_popular_credit_line_df):,}")

まずは提供者名、ジャンルでグループ化して作品点数を出力します。

In [ ]:
met_object_groupby_credit_line = met_object_popular_credit_line_df.groupby(["credit_line_name", "Object Name"]) \
                                 .size()

met_object_groupby_credit_line

横持ちになるよう、`unstack()`で各ジャンル名を列にします。

In [ ]:
credit_line_object_name_pivot_table = met_object_groupby_credit_line.unstack()

credit_line_object_name_pivot_table

ちなみに元に戻すには`stack()`です。

In [ ]:
credit_line_object_name_pivot_table.stack()

また、該当する値がない場合に`NaN`となってしまうので、`fillna()`で0をセットします。

In [ ]:
credit_line_object_name_pivot_table.fillna(0, inplace=True)

credit_line_object_name_pivot_table

それから数値が`float`になっているので、一応`int`にしておきます。  
`NaN`があるとエラーになるので、`NaN`をなくしてから変換しましょう。

これでクロス集計表は完成です。

In [ ]:
credit_line_object_name_pivot_table = credit_line_object_name_pivot_table.astype(int)

credit_line_object_name_pivot_table

実は、`pd.crosstab()`を使えば、`groupby()`以降をやらなくても簡単に表を作れてしまいます。

作成後、上の表と見比べてください。

In [ ]:
### チャレンジしてみましょう！！ ###

さらには`pd.pivot_table()`を使うこともできます。

`pd.pivot_table()`は回数だけでなく、任意の関数で数値計算をすることができます。  
今回の場合は回数ですので`aggfunc`に`len()`を指定しています。

In [ ]:
### チャレンジしてみましょう！！ ###

最後にグラフでデータを表示してみましょう。  
どんなグラフが最適でしょうか？



In [ ]:
palette = sns.color_palette('Set2', len(credit_line_object_name_pivot_table.columns))

# 配列に数値を積み上げていく。
left = np.zeros(len(credit_line_object_name_pivot_table.index))

fig, ax = plt.subplots(figsize=(16,8))

for i, column in enumerate(credit_line_object_name_pivot_table.columns):
    sns.barplot(x=credit_line_object_name_pivot_table[column], \
                     y=credit_line_object_name_pivot_table.index, \
                     color=palette[i], \
                     label=column, \
                     left=left)

    left += list(credit_line_object_name_pivot_table[column])

ax.legend(loc="upper left", bbox_to_anchor=(1,1))

### 4. 作品画像の取得

ここでは芸術家を１人ピックアップして作品の画像を取得し、年代順に並べて作風の変遷を見てみましょう。

作品画像の取得にはWebスクレイピングをおこないますので、まずは対象のページでスクレイピングが可能であるかを検証します。

作品データからランダムに１点選び、`Link Resource`列にあるURLを取得します。  
そのURLのページが実在するか確認してください。

In [ ]:
sample_link = met_object_df.sample()['Link Resource'].values[0] # sample()なので、実行するたびにリンクが変わります。

print(sample_link)

次にページのHTMLコードを取得してみましょう。

In [ ]:
response = requests.get(sample_link)

print("Status Code:", response.status_code)
response.text

ステータスコードは200ですが

> Request unsuccessful. Incapsula incident ID: 434000520055295231-179312307656068356

とあるので、ページを正しく取得できていないようです。

おそらくはプログラムを使った機械的な大量アクセスを防ぐ仕組みがあるのでしょう。  
（ここで数ページをアクセスする分には特に問題ないと思いますが。）

ブラウザを偽装したヘッダーを追加しても失敗します。

In [ ]:
headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
response = requests.get(sample_link, headers=headers)

print("Status Code:", response.status_code)
response.text

そこでChromeブラウザ経由でページにアクセスするようにしてみます。

以下の関数を定義します。

In [ ]:
def get_page_source_with_chrome(url):
    """指定のURLのHTMLコードをChromeブラウザ経由で取得。

    Args:
        url (str): ページのURL。
    Returns:
        str: HTMLソース。
    """

    browser = None

    if which('chromedriver') is None:
        # chromedriverをlibディレクトリに配置した場合

        driver_path = glob.glob('lib/chromedriver*')[0] # 拡張子がない場合も取得
        try:
            # Selenium 4
            chrome_service = webdriver.chrome.service.Service(executable_path=driver_path)
            browser = webdriver.Chrome(service=chrome_service)
        except TypeError:
            # Selenium 3
            browser = webdriver.Chrome(executable_path=driver_path)
        
    else:
        # chromedriverがサーバーにインストールされている場合

        options = webdriver.ChromeOptions()
        options.add_argument("--no-sandbox") # rootユーザーでも実行可    
        options.add_argument("--headless")   # ブラウザ画面を表示しない
        
        try:
            # Selenium 4
            chrome_service = webdriver.chrome.service.Service()
            browser = webdriver.Chrome(service=chrome_service, options=options)
        except TypeError:
            # Selenium 3
            browser = webdriver.Chrome(options=options)
        
    browser.get(url)
    source = browser.page_source
    browser.quit()

    return source

実行してみましょう。

In [ ]:
html = get_page_source_with_chrome(sample_link)

html[:2_000] # 長いので2,000文字だけ表示。

今度は正しくHTMLコードが取れているようです。

ソースコード中にある以下のようなタグを`id="artwork__image"`を足がかりにして見つけ出し、`src`属性にある画像のURLを取得します。


```
<img id="artwork__image" class="artwork__image gtm__artwork__image" src="https://collectionapi.metmuseum.org/api/collection/v1/iiif/186106/431502/main-image" alt="Tazza, Glass, Italian, Venice (Murano) " itemprop="contentUrl">
```

In [ ]:
soup = BeautifulSoup(html, "html.parser")
img = soup.find(id="artwork__image")
if img:
    src = img.attrs['src']
    display(Image(url=src, width=400))
else:
    print("Image not available.")

それでは、芸術家をピックアップし、その芸術家の芸術作品群の画像を取得していきます。

まず、以下のような条件で芸術家をピックアップします。
1. 重要作品が存在する芸術家であること。
2. 作品が絵画であること。
3. 作者不明（Anonymous）ではないこと。

In [ ]:
met_object_df['Artist Display Name'].unique()

In [ ]:
artist_with_multiple_hilight_works_df = met_object_df[ \
    met_object_df['Is Highlight'] & \
    (met_object_df['Object Name'] == 'Drawing') & \
    ~met_object_df['Artist Display Name'].str.contains('Anonymous', case=False, na=True)] \
.groupby(['Artist Display Name']) \
.agg(count=('Object Number','count')) \
.query("count > 0") \
.sort_values('count', ascending=False)

artist_with_multiple_hilight_works_df

抽出した芸術家の中からランダムに一人選択後、作品画像を取得して年代順に並べます。

どんなことが読み取れますか？

In [ ]:
# 作品画像の取得数最大値
n_max_drawing = 5

# 芸術家を選択
#pickup_artist = "Georgia O'Keeffe"
pickup_artist = artist_with_multiple_hilight_works_df.sample().index.values[0]

# 作品データを抽出
pickup_artist_work_df = met_object_df[
                        (met_object_df['Artist Display Name'] == pickup_artist) & \
                        (met_object_df['Object Name'] == 'Drawing')] \
                        .sort_values('Object End Date') \
                        .reset_index()

print(pickup_artist)
print("")

for i, work in pickup_artist_work_df.iterrows():
    
    # 最大値に達したら終了
    if i == n_max_drawing:
        break

    print(f"{i+1}. {work['Title']}")
    print(f"{work['Object End Date']}年")
    print(work['Link Resource'])

    # 作品画像を取得して表示
    html = get_page_source_with_chrome(work['Link Resource'])
    soup = BeautifulSoup(html, "html.parser")
    img = soup.find(id="artwork__image")
    if img:
        src = img.attrs['src']
        display(Image(url=src, width=400))
    else:
        print("Image not available.")